In [1]:
import pandas as pd
import numpy as np
import re
from unidecode import unidecode
from functools import reduce
import ast
from unidecode import unidecode
from sklearn.preprocessing import MultiLabelBinarizer

In [2]:
pd.set_option("display.max_columns", None)
pd.set_option('display.max_rows',None)
pd.set_option('display.float_format', lambda x: '%.2f' % x)
pd.set_option('display.max_colwidth', None)
# Ajustar la configuración para mostrar más filas 
#pd.set_option('display.max_rows', 500)

#inicializo la lista to replace
to_replace=['Null','Nan','nan','NULL',None]

In [3]:
dfbarrios= pd.read_excel('../02-datos-limpios/distr_barrios_codpostal.xlsx')

In [ ]:
#dfbarrios= pd.read_excel('C:/Mas/Dataset_Proyecto/Ficheros_Ventas/Unidos/distr_barrios_codpostal.xlsx')

In [5]:
dfi= pd.read_csv('./data/df_Indomio_paraunirenero.csv',index_col=False)
dfunion= pd.read_csv('./data/20241231_Union_Sil_Ale.csv',index_col=False)
#dfunion=pd.read_csv('./data/20250111_Union_Sil_Ale_pisosmadrid.csv',index_col=False)
dfpm= pd.read_csv('./data/dfpisosmadrid_paraunir.csv',index_col=False)
dfredpiso= pd.read_csv('./data/red_pisoparaunir.csv',index_col=False)

In [ ]:
#14012024 juntar estos ficheros

#dfi.to_csv("dfi1.csv", index=False)
#dfunion.to_csv("dfunionv1.csv", index=False)
#dfpm.to_csv("dfpmv1.csv", index=False)
#dfredpiso.to_csv("dfredpisov1.csv", index=False)
#dfi= pd.read_csv('C:/Mas/Dataset_Proyecto/Ficheros_Ventas/Unir_gilberto/dfi1.csv',index_col=False)
#dfunion= pd.read_csv('C:/Mas/Dataset_Proyecto/Ficheros_Ventas/Unir_gilberto/dfunionv1.csv',index_col=False)
#dfpm= pd.read_csv('C:/Mas/Dataset_Proyecto/Ficheros_Ventas/Unir_gilberto/dfpmv1.csv',index_col=False)
#dfredpiso= pd.read_csv('C:/Mas/Dataset_Proyecto/Ficheros_Ventas/Unir_gilberto/dfredpisov1.csv',index_col=False)

In [107]:
#14012024 juntar estos ficheros

dfi.to_csv("dfi1.csv", index=False)
dfunion.to_csv("dfunionv1.csv", index=False)
dfpm.to_csv("dfpmv1.csv", index=False)
dfredpiso.to_csv("dfredpisov1.csv", index=False)


In [5]:
# Aqui vamos a filtrar la informacion de tipo_fachada y la dividire en dos columnas donde se veran los valores de fachada y suelo separados
dfredpiso["tipo_fachada"]= np.nan

def asignar_fachada(df):
    # Recorrer cada fila de la columna Tipo de suelo/fachada
    for idx, fila in df.iterrows():
        tipo_suelo_fachada = str(fila['tipo_suelo'])  # Asegurarse de que sea un string
        # Convertir el texto a minúsculas para la búsqueda sin importar mayúsculas
        if 'fachada' in tipo_suelo_fachada.lower():
            # Si encuentra la palabra 'fachada', asignar todo el texto a la columna tipo_fachada
            df.at[idx, 'tipo_fachada'] = tipo_suelo_fachada
            # Reemplazar el valor original en Tipo de suelo/fachada por NaN
            df.at[idx, 'tipo_suelo'] = np.nan
    return df



In [6]:
dfredpiso['web']='redpiso'
dfredpiso['fecha_descarga']='31/10/2024'
# Llamar a la función pasándole tu dataframe
dfredpiso= asignar_fachada(dfredpiso)
#renombrar titulo a title en redpiso
dfredpiso.rename(columns={'titulo': 'title', 'conserje':'porteria','antiguedad':'ano_construccion','emisiones_CO2':'emisiones_co2'}, inplace=True)
dfredpiso["zona"] = dfredpiso["nombre_oficina"].str.replace("^REDPISO", "", regex=True)
dfunion.rename(columns={'read_all': 'descripcion_extendida'}, inplace=True)
dfi.rename(columns={'read_all': 'descripcion_extendida','acceso_minusv':'acceso_minusvalido'}, inplace=True)
celiminar=['url_imagen','dormitorios']
dfredpiso.drop(columns=celiminar,inplace=True)
dfredpiso.rename(columns={'cantidad_dormitorios': 'dormitorios'}, inplace=True)
celiminar=['title2','ciudad']
dfi.drop(columns=celiminar,inplace=True)

celiminar=['Unnamed: 0']
dfpm.drop(columns=celiminar,inplace=True)


#coleliminar=['salon']
#dfunion.drop(dolumns=celiminar,inplace=True)
dfi['certificado_energetico'] = dfi['letrace'].apply(lambda x: 'Disponible' if pd.notna(x) and x != '' else 'No indicado')
#en la columna antiguedad de redpiso_pisosmadrid2 quitar el texto año Ano de construccion del valor
dfredpiso['ano_construccion']=dfredpiso['ano_construccion'].str.replace('Ano de construccion','')
dfredpiso['ano_construccion']=dfredpiso['ano_construccion'].str.replace('Antiguedad','')
# dfredpiso["plantas"]=dfredpiso['plantas'].str.replace('Plantas','')
# Crear la nueva columna 'exterior' y asignar el valor 1 cuando 'vista zona' sea 'Exterior'
# dfredpiso=dfredpiso.drop("plantas",axis=1)

dfredpiso['exterior'] = dfredpiso['vista_zona'].apply(lambda x: 1 if x == 'Exterior' else np.nan)
# redpiso_pisosmadrid2=redpiso_pisosmadrid2.drop("vista zona",axis=1)



C:\Users\extas\AppData\Local\Temp\ipykernel_12916\3858861594.py:11: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Tipo de fachada Ladrillo' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.at[idx, 'tipo_fachada'] = tipo_suelo_fachada


| Column                                    | Redpiso | pm | indomio | Union  |
|:------------------------------------------|:------|:------|:------|:------|
| EUR/m2                                    | ❌    | ❌    | ✔️    | ✔️    |
| acceso_minusvalido                        | ✔️    | ❌    | ❌    | ✔️    |
| agua_caliente                             | ✔️    | ❌    | ❌    | ❌    |
| aire_acondicionado                        | ✔️    | ✔️    | ✔️    | ✔️    |
| alarma                                    | ✔️    | ✔️    | ❌    | ✔️    |
| alquiler_opcion_a_compra                  | ❌    | ❌    | ✔️    | ✔️    |
| amueblado                                 | ✔️    | ✔️    | ✔️    | ✔️    |
| ano_construccion                          | ❌    | ❌    | ✔️    | ✔️    |
| antiguedad                                | ✔️    | ❌    | ❌    | ❌    |
| armario_empotrado                         | ❌    | ❌    | ✔️    | ❌    |
| ascensor                                  | ✔️    | ❌    | ✔️    | ✔️    |
| balcon                                    | ❌    | ❌    | ✔️    | ✔️    |
| banos                                     | ✔️    | ✔️    | ✔️    | ✔️    |
| barrio                                    | ✔️    | ✔️    | ✔️    | ✔️    |
| buhardilla                                | ❌    | ❌    | ✔️    | ❌    |
| cal_energetica                            | ✔️    | ❌    | ❌    | ❌    |
| calefaccion                               | ✔️    | ✔️    | ✔️    | ✔️    |
| calle                                     | ❌    | ❌    | ✔️    | ✔️    |
| campo12                                   | ✔️    | ❌    | ❌    | ❌    |
| cancha_tenis                              | ❌    | ❌    | ✔️    | ✔️    |
| cantidad_armarios                         | ✔️    | ❌    | ❌    | ❌    |
| cantidad_visitas                          | ✔️    | ❌    | ❌    | ❌    |
| carpinteria_exterior_doble_vidrio/madera  | ❌    | ❌    | ✔️    | ✔️    |
| carpinteria_exterior_doble_vidrio/metal   | ❌    | ❌    | ✔️    | ✔️    |
| carpinteria_exterior_doble_vidrio/pvc     | ❌    | ❌    | ✔️    | ✔️    |
| carpinteria_exterior_triple_vidrio/madera | ❌    | ❌    | ✔️    | ✔️    |
| carpinteria_exterior_triple_vidrio/metal  | ❌    | ❌    | ✔️    | ✔️    |
| carpinteria_exterior_vidrio/madera        | ❌    | ❌    | ✔️    | ✔️    |
| carpinteria_exterior_vidrio/metal         | ❌    | ❌    | ✔️    | ✔️    |
| carpinteria_exterior_vidrio/pvc           | ❌    | ❌    | ✔️    | ✔️    |
| certificado_energetico                    | ❌    | ❌    | ❌    | ✔️    |
| chimenea                                  | ❌    | ✔️    | ✔️    | ✔️    |
| cocina                                    | ❌    | ❌    | ❌    | ✔️    |
| cocina_1                                  | ❌    | ❌    | ✔️    | ❌    |
| cocina_equipada                           | ✔️    | ❌    | ❌    | ❌    |
| cod_barrio                                | ✔️    | ✔️    | ✔️    | ✔️    |
| cod_barrio_2                              | ✔️    | ✔️    | ❌    | ❌    |
| cod_distrito                              | ✔️    | ✔️    | ✔️    | ✔️    |
| codigo_Postal                             | ✔️    | ✔️    | ❌    | ❌    |
| consumo_energia                           | ✔️    | ❌    | ❌    | ❌    |
| consumoce_ano                             | ❌    | ❌    | ✔️    | ✔️    |
| derecho_superficie                        | ❌    | ❌    | ✔️    | ❌    |
| descripcion_extendida                     | ✔️    | ✔️    | ✔️    | ✔️    |
| descuento                                 | ❌    | ❌    | ✔️    | ✔️    |
| detalle                                   | ❌    | ❌    | ❌    | ✔️    |
| disponibilidad                            | ❌    | ❌    | ✔️    | ✔️    |
| distrito                                  | ✔️    | ✔️    | ✔️    | ✔️    |
| dormitorios                               | ✔️    | ✔️    | ✔️    | ✔️    ||
| emisiones_co2                             | ❌    | ❌    | ✔️    | ✔️    |
| estado                                    | ✔️    | ❌    | ✔️    | ✔️    |
| exterior                                  | ✔️    | ❌    | ✔️    | ✔️    |
| fecha_descarga                            | ✔️    | ✔️    | ❌    | ✔️    |
| fecha_publicacion                         | ✔️    | ✔️    | ❌    | ❌    |
| fibra_optica                              | ❌    | ❌    | ✔️    | ✔️    |
| garaje                                    | ✔️    | ✔️    | ✔️    | ✔️    |
| gastos_comunidad                          | ✔️    | ❌    | ✔️    | ❌    |
| habitaciones                              | ❌    | ❌    | ✔️    | ✔️    |
| hidromasaje                               | ❌    | ❌    | ✔️    | ❌    |
| inmueble_ingresos                         | ❌    | ❌    | ✔️    | ✔️    |
| instalacion_tv_centralizada               | ❌    | ❌    | ✔️    | ✔️    |
| instalacion_tv_individual                 | ❌    | ❌    | ✔️    | ✔️    |
| interior                                  | ❌    | ❌    | ✔️    | ✔️    |
| interior_y_exterior                       | ❌    | ❌    | ✔️    | ✔️    |
| jardin                                    | ❌    | ✔️    | ✔️    | ✔️    |
| letrace                                   | ❌    | ❌    | ✔️    | ✔️    |
| lujosa                                    | ❌    | ❌    | ✔️    | ✔️    |
| m2_constr                                 | ✔️    | ✔️    | ✔️    | ✔️    |
| m2_utiles                                 | ✔️    | ❌    | ❌    | ❌    |
| n.visitas                                 | ❌    | ✔️    | ❌    | ❌    |
| nombre_oficina                            | ✔️    | ❌    | ❌    | ❌    |
| nuda_propiedad                            | ❌    | ❌    | ✔️    | ✔️    |
| num_pisos                                 | ✔️    | ❌    | ❌    | ❌    |
| num_plantas                               | ✔️    | ❌    | ❌    | ❌    |
| orientacion                               | ✔️    | ❌    | ❌    | ✔️    |
| orientacion_este                          | ❌    | ❌    | ✔️    | ❌    |
| orientacion_norte                         | ❌    | ❌    | ✔️    | ❌    |
| orientacion_norte,_sur                    | ❌    | ❌    | ✔️    | ❌    |
| orientacion_oeste                         | ❌    | ❌    | ✔️    | ❌    |
| orientacion_sur                           | ❌    | ❌    | ✔️    | ❌    |
| orientacion_sur,_este                     | ❌    | ❌    | ✔️    | ❌    |
| piscina                                   | ✔️    | ✔️    | ✔️    | ✔️    |
| planta                                    | ❌    | ❌    | ✔️    | ✔️    |
| plantas_edificio                          | ❌    | ❌    | ✔️    | ✔️    |
| porteria                                  | ✔️    | ❌    | ✔️    | ✔️    |
| porton_electrico                          | ❌    | ❌    | ✔️    | ✔️    |
| precio                                    | ✔️    | ✔️    | ✔️    | ✔️    |
| puerta_blindada                           | ❌    | ❌    | ✔️    | ✔️    |
| suelos_ceramica/porcelanato               | ❌    | ❌    | ✔️    | ✔️    |
| suelos_de_madera                          | ❌    | ❌    | ✔️    | ✔️    |
| suelos_gres/ceramica                      | ❌    | ❌    | ✔️    | ✔️    |
| suelos_madera_maciza                      | ❌    | ❌    | ✔️    | ✔️    |
| superficie_construida                     | ✔️    | ✔️    | ✔️    | ✔️    |
| superficie_util                           | ✔️    | ❌    | ✔️    | ✔️    |
| terraza                                   | ✔️    | ✔️    | ✔️    | ✔️    |
| tipo_anuncio                              | ❌    | ❌    | ✔️    | ✔️    |
| tipo_inmueble                             | ✔️    | ✔️    | ✔️    | ✔️    |
 tipo_suelo
| trastero                                  | ❌    | ❌    | ✔️    | ✔️    ||
| videoportero                              | ❌    | ❌    | ✔️    | ✔️    |
| zonas_verdes                              | ❌    | ❌    | ✔️    | ✔️    |



In [7]:
#creo la columna tiene armario
#dfi['tiene_armario'] = dfi['armarios'].apply(lambda x: 1 if pd.notna(x) and x != '' else np.nan)


poner zona en redpiso
falta calle en pm y redpiso
descripcion_extendida redpido y pm
estado falta en pm
antiguedad en redpiso y ano construccion en indomio y unidos, falta en pisos madrid
revisar cocinaequipada,cocina y cocina_1




In [49]:
dfb= pd.read_feather('C:/Mas/Dataset_Proyecto/Ficheros_Ventas/Unidos/df_distritos.ftr')

In [9]:
dfb.columns

Index(['cod_distrito', 'distrito', 'cod_barrio', 'barrio', 'distrito_n',
       'barrio_n'],
      dtype='object')

In [10]:
dfb

,cod_distrito,distrito,cod_barrio,barrio,distrito_n,barrio_n
21,1,Centro,11,Palacio,centro,palacio
22,1,Centro,12,Embajadores,centro,embajadores
23,1,Centro,13,Cortes,centro,cortes
24,1,Centro,14,Justicia,centro,justicia
25,1,Centro,15,Universidad,centro,universidad
26,1,Centro,16,Sol,centro,sol
27,2,Arganzuela,21,Imperial,arganzuela,imperial
28,2,Arganzuela,22,Acacias,arganzuela,acacias
29,2,Arganzuela,23,Chopera,arganzuela,chopera
30,2,Arganzuela,24,Legazpi,arganzuela,legazpi


In [11]:
#creo la columna tiene armario
#dfpisos['tiene_armario'] = dfpisos['armarios'].apply(lambda x: 1 if pd.notna(x) and x != '' else np.nan)

In [42]:

def obtener_valores_por_columna(df, columna,valor):
    """
    Devuelve los valores de las demás columnas de la fila donde la columna especificada tiene el valor dado.
    
    :param df: DataFrame de pandas
    :param columna: Nombre de la columna para buscar el valor
    :param valor: Valor único para buscar en la columna
    :return: Diccionario con los valores de las demás columnas
    """
    fila = df.loc[df[columna] == valor]
    if not fila.empty:
        return fila.iloc[0].to_dict()
    else:
        return {}




In [43]:
def procesar_distritos_nulos(df, dfb,coldf,nombrecolbarrios,col):
    """
    Procesa las filas del DataFrame donde 'distrito' es nulo y devuelve 'cod_barrio', 'cod_distrito' y 'distrito'.
    
    :param df: DataFrame de pandas con los datos a procesar
    :param dfbarrios: DataFrame de referencia para obtener los valores
    :return: DataFrame con las columnas 'cod_barrio', 'cod_distrito' y 'distrito'
    """
 
    filas_nulas = df[df['distrito'].isna() & df[coldf].notna()] 
 
    for idx, fila in filas_nulas.iterrows(): 
        
        valores = obtener_valores_por_columna(dfb, nombrecolbarrios, fila[coldf]) 
        print(valores)
        if valores:
             df.at[idx, col] = valores.get(col) 
             df.at[idx, 'cod_distrito'] = valores.get('cod_distrito') 
             df.at[idx, 'distrito'] = valores.get('nombre_distrito')






In [103]:
dfb['cod_barrio'] = dfb['cod_barrio'].astype(int)

In [ ]:
procesar_distritos_nulos(dfunion, dfb,"cod_barrio","cod_barrio","barrio")

In [44]:
# Función para normalizar texto (eliminar acentos y convertir a minúsculas)
def normalizar_texto(texto):
 return unidecode(texto).lower()

In [45]:
def ponerMay(df,campo):
    df[campo]=dfredpiso[campo].str.strip().str.capitalize()

In [16]:
ponerMay(dfredpiso,'barrio')
ponerMay(dfredpiso,'distrito')
ponerMay(dfunion,'barrio')
ponerMay(dfunion,'distrito')
ponerMay(dfi,'barrio')
ponerMay(dfi,'distrito')
ponerMay(dfpm,'barrio')
ponerMay(dfpm,'distrito')


In [17]:
#completo distritos nulos con barrio informado
procesar_distritos_nulos(dfunion, dfbarrios)
procesar_distritos_nulos(dfredpiso, dfbarrios)
procesar_distritos_nulos(dfi, dfbarrios)
procesar_distritos_nulos(dfpm, dfbarrios)

In [46]:
def asignar_zona_distrito_barrio(row):
    if row['zona'] == 'Sanchinarro':
        row['distrito'] = 'Hortaleza'
        row['barrio'] = 'Valdefuentes'
    if row['zona'] == 'Malasana':
        row['distrito'] = 'Centro'
        row['barrio'] = 'Malasaña'
    if row['zona'] == 'San Andrés':
        row['distrito'] = 'Villaverde'
        row['barrio'] = 'VillaverdeAlto,CascoHistóricodeVillaverde'
    if row['zona'] == 'Virgen del Cortijo':
        row['distrito'] = 'Hortaleza'
        row['barrio'] = 'Valdefuentes'
    if row['zona'] == 'Montecarmelo':
        row['distrito'] = 'Fuencarral-El Pardo'
        row['barrio'] = 'Mirasierra'

    return row

In [19]:
#dfpm.loc[dfpm['distrito'].isnull(), ['distrito', 'barrio']] = dfpm.loc[dfpm['distrito'].isnull()].apply(asignar_zona_distrito_barrio, axis=1)[['distrito', 'barrio']]


In [20]:
dfunion.loc[dfunion['distrito'].isnull(), ['distrito', 'barrio']] = dfunion.loc[dfunion['distrito'].isnull()].apply(asignar_zona_distrito_barrio, axis=1)[['distrito', 'barrio']]
dfredpiso.loc[dfredpiso['distrito'].isnull(), ['distrito', 'barrio']] = dfredpiso.loc[dfredpiso['distrito'].isnull()].apply(asignar_zona_distrito_barrio, axis=1)[['distrito', 'barrio']]
dfi.loc[dfi['distrito'].isnull(), ['distrito', 'barrio']] = dfi.loc[dfi['distrito'].isnull()].apply(asignar_zona_distrito_barrio, axis=1)[['distrito', 'barrio']]
dfpm.loc[dfpm['distrito'].isnull(), ['distrito', 'barrio']] = dfpm.loc[dfpm['distrito'].isnull()].apply(asignar_zona_distrito_barrio, axis=1)[['distrito', 'barrio']]

In [56]:
def asignarbarriodistritodesdezona(dfpm):
    # Filtrar las filas donde 'distrito' o 'barrio' son nulos
    #filas_nulas = dfpm[dfpm['distrito'].isna() | dfpm['barrio'].isna()]
    #filas_nulas = dfpm[dfpm['barrio'].isna()]
    filas_nulas = dfpm[dfpm['cod_barrio']==0]
    for idx, fila in filas_nulas.iterrows():
        zona = fila['zona']
        
        # Verifica si zona no es nulo antes de procesarlo
        if pd.notnull(zona):
            bar = normalizar_texto(zona.split('(')[0].strip().replace('/', '')).strip()
            #bar = normalizar_texto(zona.split('(')[0].strip().replace('/', '').replace(' ', '')).strip()

            
            # Mapeo de nombres de barrios
            mapeo_barrios = {
                'salvador': 'elsalvador',
                'san andres': 'villaverdealto,cascohistoricodevillaverde',
                'sanandres': 'villaverdealto,cascohistoricodevillaverde',
                'villaverdealto': 'villaverdealto,cascohistoricodevillaverde',
                'villaverde alto': 'villaverdealto,cascohistoricodevillaverde',
                'palos de moguer': 'palosdemoguer',
                'virgen del cortijo-manoteras': 'valdefuentes',
                'sanchinarro': 'valdefuentes',
                'montecarmelo': 'mirasierra',
                'ambroz': 'cascohistoricodevicalvaro',
                'congosto': 'cascohistoricodevallecas',
                'puente de vallecas': 'puentedevallecas',
                'villaverde bajo': 'losrosales',
                'malasana': 'universidad',
                'pau carabanchel': 'buenavista',
                'pau de carabanchel': 'buenavista',
                'bernabeu': 'hispanoamerica',
                'orense': 'cuatrocaminos',
                'arturo soria': 'concepcion',
                'chueca': 'justicia',
                'lavapies': 'embajadores',
                'las mercedes': 'rejas',
                'casco historico de barajas': 'cascohistoricodebarajas',
                'conde ordaz': 'piovera',
                'san blas': 'arcos',
                'campo de las naciones': 'corralejos',
                'puerta de hierro': 'aravaca',
                'ventilla': 'almenara',
                'gran via': 'sol',
                'ensanchedevallecas': 'ensanchedevallecas',
                'fuencarral': 'valverde',
                'las tablas': 'valverde',
                'buena vista': 'buenavista',
                'principe pio': 'arguelles',
                'los berrocales': 'losberrocales',
                'plaza eliptica': 'abrantes',
                'palomeras': 'palomerasbajas',
                'carabanchel alto': 'buenavista',
                'carmenes': 'loscarmenes',
                'los angeles': 'angeles',
                'ciudaddelosangeles':'angeles',
                'valdebebas': 'valdefuentes',
                'virgendelcortijo': 'valdefuentes',
                'carabancheloporto':'opanel',
                'carabanchel eugenia de montijo':'vistaalegre',
                'carabanchelurgel':'opanel',
                'vallecaspena prieta':'sandiego',
                'san bernardo  malasana  chamberi':'universidad',
                'marques de vadillomadrid rio':'comillas',
                'san diego':'sandiego',
                'lalatina':'embajadores',
                'barriodesalamanca':'salamanca',
                'moncloa':'moncloa-aravaca'
            }
            
            print(f"{bar}")
            # Asignar el nombre del barrio mapeado
            bar = mapeo_barrios.get(bar, bar)
            
            # Para los berrocales como es un barrio nuevo, lo creo aunque no exista oficialmente
            if bar == 'losberrocales':
                dfpm.at[idx, 'cod_barrio'] = 195
            
            valores = obtener_valores_por_columna(dfb, 'barrio_n', bar)
            if valores:
                if pd.isna(dfpm.at[idx, 'barrio']):
                    dfpm.at[idx, 'barrio'] = valores.get('barrio')
                if pd.isna(dfpm.at[idx, 'cod_barrio']):
                    dfpm.at[idx, 'cod_barrio'] = valores.get('cod_barrio')
                if pd.isna(dfpm.at[idx, 'cod_distrito']):
                    dfpm.at[idx, 'cod_distrito'] = valores.get('cod_distrito')
                if pd.isna(dfpm.at[idx, 'distrito']):
                    dfpm.at[idx, 'distrito'] = valores.get('distrito')
                dfpm.at[idx, 'zona'] = bar

            valores = obtener_valores_por_columna(dfb, 'distrito_n', bar)
            if valores:
                if pd.isna(dfpm.at[idx, 'cod_distrito']):
                    dfpm.at[idx, 'cod_distrito'] = valores.get('cod_distrito')
                if pd.isna(dfpm.at[idx, 'distrito']):
                    dfpm.at[idx, 'distrito'] = valores.get('distrito')
                dfpm.at[idx, 'zona'] = bar

    return dfpm


In [ ]:
dfunion=asignarbarriodistritodesdezona(dfunion)
dfredpiso=asignarbarriodistritodesdezona(dfredpiso)
dfi=asignarbarriodistritodesdezona(dfi)
dfpm=asignarbarriodistritodesdezona(dfpm)

In [36]:
dfredpiso['barrio'] = dfredpiso['barrio'].str.strip().str.capitalize()
dfredpiso['distrito'] = dfredpiso['distrito'].str.strip().str.capitalize()
dfunion['barrio'] = dfunion['barrio'].str.strip().str.capitalize()
dfunion['distrito'] = dfunion['distrito'].str.strip().str.capitalize()

In [26]:
dfpm.loc[dfpm['zona'] == 'Vallecas', 'distrito'] = 'Puente de Vallecas'
dfpm.loc[dfpm['zona'] == 'Moncloa', 'distrito'] = 'Moncloa-Aravaca'

In [27]:
dfpm.loc[dfpm['zona'] == 'Vallecas', 'distrito'] = 'Puente de Vallecas'
dfpm.loc[dfpm['zona'] == 'Moncloa', 'distrito'] = 'Moncloa-Aravaca'

In [28]:
dfbarrios["nombre_distrito"] = dfbarrios["nombre_distrito"].apply(lambda x: x.strip())


In [ ]:
dfunion[dfunion['distrito'].isna() & dfunion['barrio'].isna()]

In [73]:
# Asignar el valor a las filas de la 7765 a la 7771 en la columna 'zona'
dfunion.loc[7765:7771, 'zona'] = "zona centro histórico vicalvaro"
dfunion.loc[7765:7771, 'cod_barrio']=191

In [ ]:

dfunion.iloc[7765:7771]

In [55]:
def procesar_distritos_nulos(df, dfbarrios):
    """
    Procesa las filas del DataFrame donde 'distrito' es nulo y devuelve 'cod_barrio', 'cod_distrito' y 'distrito'.
    
    :param df: DataFrame de pandas con los datos a procesar
    :param dfbarrios: DataFrame de referencia para obtener los valores
    :return: DataFrame con las columnas 'cod_barrio', 'cod_distrito' y 'distrito'
    """
 
    filas_nulas = df[df['distrito'].isna() & df['cod_barrio'].notna()] 
    for idx, fila in filas_nulas.iterrows(): 
        valores = obtener_valores_por_columna(dfbarrios, 'nombre_barrio', fila['cod_barrio']) 
        if valores:
             df.at[idx, 'barrio'] = valores.get('barrio') 
             df.at[idx, 'cod_distrito'] = valores.get('cod_distrito') 
             df.at[idx, 'distrito'] = valores.get('nombre_distrito')

In [50]:
#teniendo codigo de barrio, pongo el resto de columnas sino tienen valor
# Filtrar las filas donde 'distrito' o 'barrio' son nulos

def ponerdistritoconcodbarrio(df):
    filas_nulas = df[df['distrito'].isna() | df['barrio'].isna()  | df['cod_distrito'].isna()]

    for idx, fila in filas_nulas.iterrows():
    #bar = normalizar_texto(fila['ubicacion'].split()[:2])
   
        bar = (fila['distrito'])
       # print(bar)
        fila = dfbarrios.loc[dfbarrios['nombre_distrito'] == bar]
        if not fila.empty:
         valores=fila.iloc[0].to_dict()
        #print(valores)
         df.at[idx, 'cod_distrito'] = valores.get('cod_distrito')



In [57]:
procesar_distritos_nulos(dfunion, dfbarrios)

In [30]:
dfi.shape,dfpm.shape,dfunion.shape,dfredpiso.shape

((288, 78), (893, 30), (7878, 75), (1470, 54))

In [3]:
#14012024 juntar estos ficheros

#dfi.to_csv("dfi1.csv", index=False)
#dfunion.to_csv("dfunionv1.csv", index=False)
#dfpm.to_csv("dfpmv1.csv", index=False)
#dfredpiso.to_csv("dfredpisov1.csv", index=False)
dfi= pd.read_csv('C:/Mas/Dataset_Proyecto/Ficheros_Ventas/Unir_gilberto/dfi1.csv',index_col=False)
dfunion= pd.read_csv('C:/Mas/Dataset_Proyecto/Ficheros_Ventas/Unir_gilberto/dfunionv1.csv',index_col=False)
#dfunion=pd.read_csv('C:/Mas/Dataset_Proyecto/Ficheros_Ventas/Unir_gilberto/20250111_Union_Sil_Ale_pisosmadrid.csv',index_col=False)
dfpm= pd.read_csv('C:/Mas/Dataset_Proyecto/Ficheros_Ventas/Unir_gilberto/dfpmv1.csv',index_col=False)
dfredpiso= pd.read_csv('C:/Mas/Dataset_Proyecto/Ficheros_Ventas/Unir_gilberto/dfredpisov1.csv',index_col=False)

In [11]:
nuevos_nombres ={'armario_empotrado': 'tiene_armario','piscina':'tiene_piscina'
 } 
dfi = dfi.rename(columns=nuevos_nombres)

# me llego aqui

In [ ]:
#14012024 juntar estos ficheros

dfi.to_csv("dfi1.csv", index=False)
dfunion.to_csv("dfunionv1.csv", index=False)
dfpm.to_csv("dfpmv1.csv", index=False)
dfredpiso.to_csv("dfredpisov1.csv", index=False)

In [14]:
# Asignar 0 a 'dormitorios' y 'cantidad_dormitorios' cuando 'tipologia' es 'estudio'
dfredpiso.loc[dfredpiso['tipologia'] == 'Estudio', ['dormitorios', 'cantidad_dormitorios']] = 0

In [17]:
dfunion['terraza'].value_counts(dropna=False)

terraza
0.00    4088
1.00    1913
NaN     1877
Name: count, dtype: int64

In [18]:
# Reemplazar "Sí" por 1 y "No" por 0 
dfi['trastero'] = dfi['trastero'].replace({'Sí': 1, 'No': 0})
dfi['terraza'] = dfi['terraza'].replace({'Sí': 1, 'No': 0})

C:\Users\extas\AppData\Local\Temp\ipykernel_12632\3458785875.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  dfi['trastero'] = dfi['trastero'].replace({'Sí': 1, 'No': 0})
C:\Users\extas\AppData\Local\Temp\ipykernel_12632\3458785875.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  dfi['terraza'] = dfi['terraza'].replace({'Sí': 1, 'No': 0})


# comienza el merge

In [ ]:
dfunion.columns

In [ ]:
dfredpiso.columns

In [22]:
dfunion.shape,dfredpiso.shape,dfi.shape,dfpm.shape

((7878, 77), (1470, 55), (288, 79), (893, 30))

In [23]:
df_unido = pd.concat([dfunion, dfredpiso,dfi,dfpm], ignore_index=True)

In [24]:
df_unido.shape

(10529, 115)

In [ ]:
df_unido.columns.tolist()

In [ ]:
dfb

In [58]:
df_unido=asignarbarriodistritodesdezona(df_unido)

villaverdealto,cascohistoricodevillaverde
valdefuentes
valdefuentes
villaverdealto,cascohistoricodevillaverde
valverde
loscarmenes
loscarmenes
valdefuentes
villaverdealto,cascohistoricodevillaverde
valdefuentes
loscarmenes
mirasierra
angeles
valdefuentes
villaverdealto,cascohistoricodevillaverde
cascohistoricodevicalvaro
loscarmenes
valdefuentes
villaverdealto,cascohistoricodevillaverde
valdefuentes
villaverdealto,cascohistoricodevillaverde
valdefuentes
valdefuentes
valdefuentes
elsalvador
valdefuentes
valdefuentes
loscarmenes
villaverdealto,cascohistoricodevillaverde
valdefuentes
valdefuentes
elsalvador
villaverdealto,cascohistoricodevillaverde
valdefuentes
villaverdealto,cascohistoricodevillaverde
valdefuentes
valverde
mirasierra
angeles
elsalvador
villaverdealto,cascohistoricodevillaverde
valdefuentes
villaverdealto,cascohistoricodevillaverde
villaverdealto,cascohistoricodevillaverde
valdefuentes
loscarmenes
villaverdealto,cascohistoricodevillaverde
villaverdealto,cascohistoricodevi

In [39]:
df_unido.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10529 entries, 0 to 10528
Columns: 115 entries, title to n.visitas
dtypes: float64(61), object(54)
memory usage: 9.2+ MB


In [ ]:
dfb

In [74]:
dfb.reset_index(drop=True, inplace=True)

In [78]:


dfb=dfb.iloc[0:131,:]

In [98]:
print(barrio_mapping)

{11: 'Palacio', 12: 'Embajadores', 13: 'Cortes', 14: 'Justicia', 15: 'Universidad', 16: 'Sol', 21: 'Imperial', 22: 'Acacias', 23: 'Chopera', 24: 'Legazpi', 25: 'Delicias', 26: 'PalosdeMoguer', 27: 'Atocha', 31: 'Pacífico', 32: 'Adelfas', 33: 'Estrella', 34: 'Ibiza', 35: 'Jerónimos', 36: 'NiñoJesús', 41: 'Recoletos', 42: 'Goya', 43: 'FuentedelBerro', 44: 'Guindalera', 45: 'Lista', 46: 'Castellana', 51: 'ElViso', 52: 'Prosperidad', 53: 'CiudadJardín', 54: 'Hispanoamérica', 55: 'NuevaEspaña', 56: 'Castilla', 61: 'BellasVistas', 62: 'CuatroCaminos', 63: 'Castillejos', 64: 'Almenara', 65: 'Valdeacederas', 66: 'Berruguete', 71: 'Gaztambide', 72: 'Arapiles', 73: 'Trafalgar', 74: 'Almagro', 75: 'RíosRosas', 76: 'Vallehermoso', 81: 'ElPardo', 82: 'Fuentelarreina', 83: 'Peñagrande', 84: 'Pilar', 85: 'LaPaz', 86: 'Valverde', 87: 'Mirasierra', 88: 'ElGoloso', 91: 'CasadeCampo', 92: 'Argüelles', 93: 'CiudadUniversitaria', 94: 'Valdezarza', 95: 'Valdemarín', 96: 'ElPlantío', 97: 'Aravaca', 101: 'Los

In [91]:
dfb['cod_barrio'] = dfb['cod_barrio'].astype(int)
df_unido['cod_barrio'] = pd.to_numeric(df_unido['cod_barrio'], errors='coerce')  # Convertir a numérico, NaN para no válidos
df_unido['cod_barrio'] = df_unido['cod_barrio'].fillna(0).astype(int) 
df_unido['cod_barrio'] = df_unido['cod_barrio'].astype(int)


C:\Users\extas\AppData\Local\Temp\ipykernel_12632\1831268599.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfb['cod_barrio'] = dfb['cod_barrio'].astype(int)


In [99]:
# Crear un diccionario para mapear cod_barrio -> barrio
barrio_mapping = dict(zip(dfb['cod_barrio'], dfb['barrio']))
#df_unido['cod_barrio'] = df_unido['cod_barrio'].astype(str)
# Actualizar el df1 con los valores correspondientes de barrio
df_unido['barrio'] = dfb['cod_barrio'].map(barrio_mapping)

In [117]:
dfb

,cod_distrito,distrito,cod_barrio,barrio,distrito_n,barrio_n
0,1,Centro,11,Palacio,centro,palacio
1,1,Centro,12,Embajadores,centro,embajadores
2,1,Centro,13,Cortes,centro,cortes
3,1,Centro,14,Justicia,centro,justicia
4,1,Centro,15,Universidad,centro,universidad
5,1,Centro,16,Sol,centro,sol
6,2,Arganzuela,21,Imperial,arganzuela,imperial
7,2,Arganzuela,22,Acacias,arganzuela,acacias
8,2,Arganzuela,23,Chopera,arganzuela,chopera
9,2,Arganzuela,24,Legazpi,arganzuela,legazpi


In [110]:
dfb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 131 entries, 0 to 130
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   cod_distrito  131 non-null    object
 1   distrito      131 non-null    object
 2   cod_barrio    131 non-null    int32 
 3   barrio        131 non-null    object
 4   distrito_n    131 non-null    object
 5   barrio_n      131 non-null    object
dtypes: int32(1), object(5)
memory usage: 5.8+ KB


In [113]:
df_unido['cod_barrio'] = df_unido['cod_barrio'].astype(int)

In [115]:
def procesar_distritos_nulos(df, dfb):
    """
    Procesa las filas del DataFrame donde 'distrito' es nulo y devuelve 'cod_barrio', 'cod_distrito' y 'distrito'.
    
    :param df: DataFrame de pandas con los datos a procesar
    :param dfbarrios: DataFrame de referencia para obtener los valores
    :return: DataFrame con las columnas 'cod_barrio', 'cod_distrito' y 'distrito'
    """
 
    #filas_nulas = df[df['distrito'].isna() & df['cod_barrio'].notna()] 
    filas_nulas = df[df['cod_barrio'].notna()] 
    for idx, fila in filas_nulas.iterrows(): 
      #  valores = obtener_valores_por_columna(dfbarrios, 'barrio', fila['cod_barrio']) 
        valores = obtener_valores_por_columna(dfb, 'distrito', fila['cod_distrito']) 
        if valores:
            # df.at[idx, 'barrio'] = valores.get('barrio') 
            # df.at[idx, 'cod_distrito'] = valores.get('cod_distrito') 
             df.at[idx, 'distrito'] = valores.get('nombre_distrito')

In [116]:
procesar_distritos_nulos(df_unido, dfb)

In [119]:
def obtener_valores_por_columna(df, columna, valor):
    """
    Devuelve los valores de las demás columnas de la fila donde la columna especificada tiene el valor dado.
    
    :param df: DataFrame de pandas
    :param columna: Nombre de la columna para buscar el valor
    :param valor: Valor único para buscar en la columna
    :return: Diccionario con los valores de las demás columnas
    """
    fila = df.loc[df[columna] == valor]
    if not fila.empty:
        return fila.iloc[0].to_dict()  # Devuelve los valores de la fila como un diccionario
    else:
        return {}  # Si no hay coincidencias, devuelve un diccionario vacío

def procesar_barrio_y_distrito(df, dfb):
    """
    Procesa las filas del DataFrame donde 'barrio' o 'distrito' son nulos.
    Usa 'cod_barrio' para buscar valores en el DataFrame de referencia.
    
    :param df: DataFrame de pandas con los datos a procesar
    :param dfb: DataFrame de referencia para obtener los valores
    :return: DataFrame con las columnas 'barrio' y 'distrito' completadas
    """
    # Filtrar filas donde 'cod_barrio' no es nulo
    filas_a_procesar = df[df['cod_barrio'].notna()]

    for idx, fila in filas_a_procesar.iterrows():
        # Obtener valores desde el DataFrame de referencia usando 'cod_barrio'
        valores = obtener_valores_por_columna(dfb, 'cod_barrio', fila['cod_barrio'])
        
        if valores:
            # Completar las columnas 'barrio' y 'distrito' en el DataFrame original
            df.at[idx, 'barrio'] = valores.get('barrio')
            df.at[idx, 'distrito'] = valores.get('distrito')
    
    return df


In [121]:
df_unido=procesar_barrio_y_distrito(df_unido, dfb)

In [ ]:
df_unido.tail(200)

In [123]:
df_unido['cod_barrio'] = df_unido['cod_barrio'].replace(0, np.nan)

#  15012025 me llego aqui

In [126]:
df_unido.to_csv("dfunido.csv", index=False)


In [393]:
#df_union = df_union.drop(columns=['disponibilidad'])
df_union = df_union.drop(columns=['usufructo'])
df_union = df_union.drop(columns=['derecho_superficie'])
df_union = df_union.drop(columns=['buhardilla'])
df_union = df_union.drop(columns=['hidromasaje'])

In [396]:
df_union["jardin"].value_counts(dropna=False)

jardin
 NaN            6628
 Sin jardín      859
0.00             741
 Privado         244
 Comunitario     158
1.00             141
Name: count, dtype: int64

In [395]:

# Reemplazar los valores según las indicaciones
df_union['piscina'] = df_union['piscina'].replace({
    '-nan': 'NaN',
    'NaN': 'NaN',
    '-Comunitaria': 'Comunitaria',
    '-Propia': 'Privada',
    '-Con salorium': 'Comunitaria',
    '-1 piscina de adultos y 1 piscina de niños': 'Comunitaria'
})



In [397]:
df_union['tiene_jardin'] = df_union['jardin'].apply(lambda x: 1 if pd.notnull(x) and x != 'Sin jardín' else (0 if x == 'Sin jardín' else None))

In [398]:
df_union['jardin'] = df_union['jardin'].replace({
    'Jardín privado': 'Privado',
    'Jardín comunitario': 'Comunitario',
    'Jardín privado y comunitario': 'Privado'
})


In [400]:
df_union["tipologia"].value_counts(dropna=False)

tipologia
Piso           7369
Ático           330
Chalet          287
Casa            248
Apartamento     192
Dúplex          149
Estudio          88
Atico            45
Loft             24
Duplex           17
Adosado          12
Negocio           5
Pareado           3
Villa             1
Bungalow          1
Name: count, dtype: int64

In [447]:
#Reemplazar todas las tipologías que comienzan con "Casa" por "Casa" 
df_union['tipologia'] = df_union['tipologia'].replace(r'^Casa.*', 'Casa', regex=True)
df_union['tipologia'] = df_union['tipologia'].replace(r'^Apartamento.*', 'Apartamento', regex=True)
df_union['tipologia'] = df_union['tipologia'].replace(r'^Caserío.*', 'Casa', regex=True)

In [401]:

# Eliminar las filas donde 'precio' es nulo 
df_union = df_union.dropna(subset=['precio'])

In [402]:
df_union.columns

Index(['title', 'url', 'precio', 'precio_anterior', 'descuento', 'EUR/m2',
       'm2_constr', 'distrito', 'cod_distrito', 'barrio', 'cod_barrio', 'zona',
       'calle', 'detalle', 'consumoce_ano', 'letrace', 'emisiones_co2',
       'tipologia', 'propiedad_completa', 'estado', 'amueblado', 'planta',
       'habitaciones', 'dormitorios', 'banos', 'balcon', 'terraza', 'garaje',
       'trastero', 'ascensor', 'porteria', 'acceso_minusv', 'calefaccion',
       'ano_construccion', 'plantas_edificio', 'aire_acondicionado', 'jardin',
       'gastos_comunidad', 'inmueble_ingresos', 'alquiler_opcion_a_compra',
       'disponibilidad', 'nuda_propiedad', 'sup_comercial', 'tipo_inmueble',
       'tiene_armario', 'cancha_tenis',
       'carpinteria_exterior_doble_vidrio/pvc',
       'carpinteria_exterior_doble_vidrio/madera',
       'carpinteria_exterior_doble_vidrio/metal',
       'carpinteria_exterior_triple_vidrio/madera',
       'carpinteria_exterior_triple_vidrio/metal',
       'carpinteria_e

In [457]:
df_union.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7878 entries, 0 to 7881
Data columns (total 74 columns):
 #   Column                                     Non-Null Count  Dtype  
---  ------                                     --------------  -----  
 0   title                                      7878 non-null   object 
 1   url                                        7878 non-null   object 
 2   precio                                     7878 non-null   float64
 3   precio_anterior                            55 non-null     float64
 4   descuento                                  7878 non-null   object 
 5   EUR/m2                                     7860 non-null   float64
 6   m2_constr                                  7865 non-null   float64
 7   distrito                                   7878 non-null   object 
 8   cod_distrito                               7878 non-null   float64
 9   barrio                                     7757 non-null   object 
 10  cod_barrio                   

In [403]:
#Guardo la union de los dos datasets
df_union.to_csv("20250111_Union_Sil_Ale_pisosmadrid.csv",index=False)
#df_union.to_feather("20241231_Union_Sil_Ale.ftr")

In [9]:
df_union=pd.read_csv("20250111_Union_Sil_Ale_pisosmadrid.csv")


C:\Users\extas\AppData\Local\Temp\ipykernel_20884\1692700688.py:1: DtypeWarning: Columns (4,12,14,15,16,20,21,22,24,27,30,40,43,74) have mixed types. Specify dtype option on import or set low_memory=False.
  df_union=pd.read_csv("20250111_Union_Sil_Ale_pisosmadrid.csv")


In [ ]:
#en la columna antiguedad de dfr_pm quitar el texto año Ano de construccion del valor
dfr_pm['antiguedad']=dfr_pm['antiguedad'].str.replace('Ano de construccion','')
dfr_pm['antiguedad']=dfr_pm['antiguedad'].str.replace('Antiguedad','')
dfr_pm["plantas_edificio"]=dfr_pm['plantas'].str.replace('Plantas','')
# Crear la nueva columna 'exterior' y asignar el valor 1 cuando 'vista zona' sea 'Exterior'
dfr_pm['exterior'] = dfr_pm['vista zona'].apply(lambda x: 1 if x == 'Exterior' else np.nan)
dfr_pm=dfr_pm.drop("vista zona",axis=1)
dfr_pm=dfr_pm.drop("plantas",axis=1)
# Diccionario con los nuevos nombres de las columnas
#voy a suponer que la superficie es superficie construida
nuevos_nombres ={'descripcion': 'title',
'espacio_metros_cuadrados': 'm2_constr',
'm_cuadrados_utiles':'m2_utiles',
'cant_dormitorios': 'dormitorios',
'cant_banos': 'banos' ,
'condicion-estado':'estado',
'sistema_calefaccion':'calefaccion',
'antiguedad' :'ano_construccion'} 
dfr_pm = dfr_pm.rename(columns=nuevos_nombres)
#de alguna manera calefaccion estaba duplicada, quito las columnas duplicadas
dfr_pm = dfr_pm.loc[:, ~dfr_pm.columns.duplicated()]
#quito las filas con precio nulo
dfr_pm=dfr_pm.query("precio.notna()")
# Asegurarse de que todos los nombres de las columnas sean cadenas de texto 
dfr_pm.columns = dfr_pm.columns.astype(str) 
# Eliminar espacios en blanco al principio o al final de los nombres de las columnas
dfr_pm.columns = dfr_pm.columns.str.strip()